In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
# check the versions of libraries
import sys
print('Python: {}'.format(sys.version))
# scipy
import scipy
print('scipy: {}'.format(scipy.__version__))
# numpy
import numpy as np
print('numpy: {}'.format(np.__version__))
# matplotlib
import matplotlib
print('matplotlib: {}'.format(matplotlib.__version__))
# pandas
import pandas as pd 
print('pandas: {}'.format(pd.__version__))
# scikit-learn
import sklearn
print('sklearn: {}'.format(sklearn.__version__))
import tensorflow as tf
print('TensorFlow: {}'.format(tf.__version__))

Python: 3.6.11 | packaged by conda-forge | (default, Jul 23 2020, 22:18:32) 
[GCC 7.5.0]
scipy: 1.5.2
numpy: 1.19.2
matplotlib: 3.3.2
pandas: 1.1.3
sklearn: 0.23.2


/home/maguileraf23/.conda/envs/deeplift/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/maguileraf23/.conda/envs/deeplift/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/maguileraf23/.conda/envs/deeplift/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/maguileraf23/.conda/envs/

TensorFlow: 1.14.0


In [7]:
from pandas.plotting import scatter_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier #discriminative and supervised
from time import time
from sklearn.feature_selection import SelectFromModel

# Read in feature matrix (train & test)

 Train sequences and label feature matrix

In [16]:
#file containing the feat matrix of sequences for training
#tomato
input_file = 'feat_mats/train/k40_noninoc_5k_seq_kmer_feat_040322.csv'

In [17]:
#load csv file in chunks for memory 
%time chunk = pd.read_csv(input_file, header=None, chunksize=1000000)

Read csv with chunks:  0.0015160361925760904 min


In [18]:
#concatenate chunks of csv file
%time dataset = pd.concat(chunk)

Concat chunks:  0.8915443142255147 min


In [19]:
#name_column = ['label'] ##this is only for when the label and seq feat mat is together
#rename columns to number of kmer
name_column = []
for i in range(0, len(dataset.columns)):
    name_column.append("kmer" + str(i))
dataset.columns = name_column

In [20]:
dataset.tail()

,kmer0,kmer1,kmer2,kmer3,kmer4,kmer5,kmer6,kmer7,kmer8,kmer9,...,kmer16374,kmer16375,kmer16376,kmer16377,kmer16378,kmer16379,kmer16380,kmer16381,kmer16382,kmer16383
9995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9996,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
9997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9999,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
#load file with feat matrix of labels
#tomato
label_file = 'feat_mats/train/k40_noninoc_5k_label_kmer_feat_040322.csv'
#read csv file
label_df = pd.read_csv(label_file, header=None)

In [22]:
#rename column to 'label'
label = label_df.rename({0: 'label'}, axis=1)
label

,label
0,1
1,1
2,1
3,1
4,1
...,...
9995,0
9996,0
9997,0
9998,0


Additional test set

# Divide training into 80/20 (original model trained with this setting)

In [23]:
X_train, X_test = train_test_split(dataset, test_size=0.2, random_state=1)

In [29]:
y_train,y_test = train_test_split(label, test_size=0.2, random_state=1)

In [30]:
y_train.head()

,label
2694,1
5140,0
2568,1
3671,1
7427,0


# Random Forest

# Select top N important k-mers

In [32]:
#sel = SelectFromModel(RandomForestClassifier(), max_features=1500)
sel = SelectFromModel(RandomForestClassifier())
sel.fit(X_train, y_train.values.ravel())

SelectFromModel(estimator=RandomForestClassifier())

# Transform feat matrix to imp kmers

Get new training files w/subset of kmers

In [33]:
#transform feat matrix to imp kmers
selected_feat= X_train.columns[(sel.get_support())]
len(selected_feat)

2061

In [35]:
imp_kmer = list(selected_feat)
df_indx = pd.DataFrame(imp_kmer)
df_indx[0] = df_indx[0].str.replace('kmer','')

In [36]:
df_indx.head()

,0
0,0
1,1
2,3
3,7
4,12


In [280]:
df_indx.to_csv('RF_k40_noninoc_1500_imp_kmer_indx_091322.csv', index=False, header=False)

In [37]:
X_selected = X_train[X_train.columns.intersection(selected_feat)]

In [167]:
#Save feat matrix selected features
#X_selected.to_csv('tomato_k40_noninoc_10_feature_selected_seq_091222.csv', index=False, header=True)

Get new testing files w/subset of kmers

In [38]:
#X_test_selected = test_dataset[test_dataset.columns.intersection(selected_feat)]
X_test_selected = X_test[X_test.columns.intersection(selected_feat)]

In [39]:
X_test_selected.shape

(2000, 2061)

In [174]:
#Save feat matrix selected features
#X_test_selected.to_csv('tomato_culture_mix_10_feature_selected_seq_091222.csv', index=False, header=True)

# Train and test a new RF model w/important k-mers

In [40]:
RF_model = RandomForestClassifier()
%time RF_model.fit(X_selected, y_train.values.ravel())

CPU times: user 6.91 s, sys: 0 ns, total: 6.91 s
Wall time: 6.91 s


RandomForestClassifier()

In [41]:
%time RF_predictions = RF_model.predict(X_test_selected)

CPU times: user 74.1 ms, sys: 0 ns, total: 74.1 ms
Wall time: 73.4 ms


In [42]:
score = accuracy_score(y_test ,RF_predictions)

In [43]:
score

0.906

# XGBoost

In [52]:
from xgboost import XGBClassifier

In [264]:
# fit model on training seqs and labels
XGBoost_model = XGBClassifier(nthread=8)
%time XGBoost_model.fit(X_train, y_train.values.ravel())

CPU times: user 3min 51s, sys: 810 ms, total: 3min 51s
Wall time: 31.8 s


XGBClassifier(nthread=8)

In [265]:
# make predictions for test data and evaluate
%time y_pred = XGBoost_model.predict(X_test)

CPU times: user 627 ms, sys: 1 ms, total: 628 ms
Wall time: 549 ms


In [266]:
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 88.55%


In [174]:
important_feat_indx = np.argwhere(XGBoost_model.feature_importances_ > 0)

In [175]:
xgboost_imp_kmers = important_feat_indx.flatten()

In [65]:
pd.DataFrame(xgboost_imp_kmers).to_csv('xgboost_k40_noninoc_imp_kmer_indx_091322.csv', 
                                       index=False, header=False)

In [186]:
#Select features above threshold 
selection = SelectFromModel(XGBoost_model, threshold=0.000000001, prefit=True)

In [187]:
#reduce k-mers (columns) to keep only the important ones above threshold
select_X_train = selection.transform(X_train)

In [188]:
select_X_train.shape

(8000, 477)

In [189]:
select_X_test = selection.transform(X_test)

In [190]:
select_X_test.shape

(2000, 477)

In [191]:
# fit model on important kmers
XGBoost_model = XGBClassifier(nthread=8)
%time XGBoost_model.fit(select_X_train, y_train.values.ravel())

CPU times: user 5.9 s, sys: 1.02 ms, total: 5.9 s
Wall time: 758 ms


XGBClassifier(nthread=8)

In [192]:
# make predictions for test data and evaluate
%time y_pred = XGBoost_model.predict(select_X_test)

CPU times: user 14.7 ms, sys: 0 ns, total: 14.7 ms
Wall time: 6.1 ms


In [193]:
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 88.55%


# SVM

In [4]:
from sklearn.svm import SVC
from sklearn.feature_selection import RFE
import pickle

In [5]:
#load the saved SVM linear
with open('svm_lin_model_5k_tomato_k40_091922', 'rb') as f:
    svm_l = pickle.load(f)

In [13]:
#select imp kmers for prediction
sel_svm = SelectFromModel(svm_l, prefit=True)

In [24]:
#transform feat matrix to imp kmers
selected_feat_svm = X_train.columns[(sel_svm.get_support())]

6763

In [45]:
#transform the training kmer feat matrix to keep only the selected ones
X_selected_svm = X_train[X_train.columns.intersection(selected_feat_svm)]

In [46]:
#transform the test set to keep only selected kmers
X_test_selected_svm = X_test[X_test.columns.intersection(selected_feat_svm)]

In [47]:
X_test_selected_svm.shape

(2000, 6763)

In [48]:
svm_l_model = SVC(kernel ='linear')
%time svm_l_model.fit(X_selected_svm, y_train.values.ravel())

CPU times: user 2min 13s, sys: 18.7 ms, total: 2min 13s
Wall time: 2min 13s


SVC(kernel='linear')

In [49]:
%time svm_l_predictions = svm_l_model.predict(X_test_selected_svm)

CPU times: user 30.9 s, sys: 0 ns, total: 30.9 s
Wall time: 30.9 s


In [50]:
score = accuracy_score(y_test, svm_l_predictions)
score

0.894

In [51]:
#extract important kmers
imp_kmer_svm = list(selected_feat_svm)
df_indx_svm = pd.DataFrame(imp_kmer_svm)
df_indx_svm[0] = df_indx_svm[0].str.replace('kmer','')
df_indx_svm.head()

,0
0,1
1,2
2,5
3,6
4,10


In [52]:
#save imp kmer index to file
df_indx_svm.to_csv('svm_k40_noninoc_6763_imp_kmer_indx_091822.csv', index=False, header=False)